In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=3c81ec5a5919e1b0e856cbc1048b064bf3163b652eded2db96cfb339513fe0bb
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
#Creating spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
#Create Spark Dataraframe and reading data using Dataframe reader API
ecommerce_df= spark.read.format('csv').option('header',True).load("/content/ecommerce.csv")
ecommerce_df.printSchema()

root
 |-- Customer ID: string (nullable = true)
 |-- Purchase Date: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Product Price: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Total Purchase Amount: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Customer Age: string (nullable = true)
 |-- Returns: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Churn: string (nullable = true)



#Data_Preprocessing

In [4]:
#Validating the schema of the Dataframe using inferSchema function
ecommerce_df= spark.read.format('csv').option('inferSchema',True).option('header',True).load("/content/ecommerce.csv")
ecommerce_df.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+
|Customer ID|      Purchase Date|Product Category|Product Price|Quantity|Total Purchase Amount|Payment Method|Customer Age|Returns|      Customer Name|Age|Gender|Churn|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+
|      46251|2020-09-08 09:38:32|     Electronics|           12|       3|                  740|   Credit Card|          37|    0.0|Christine Hernandez| 37|  Male|    0|
|      46251|2022-03-05 12:56:35|            Home|          468|       4|                 2739|        PayPal|          37|    0.0|Christine Hernandez| 37|  Male|    0|
|      46251|2022-05-23 18:18:01|            Home|          288|       2|                 3196|        PayPal|          37|    0.0|Christine Hernandez| 37|

In [5]:
#Checking the schema after validation
ecommerce_df.printSchema()

root
 |-- Customer ID: integer (nullable = true)
 |-- Purchase Date: timestamp (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Product Price: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Total Purchase Amount: integer (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Customer Age: integer (nullable = true)
 |-- Returns: double (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
#Describing the properties of the data
ecommerce_df.describe().show()

+-------+------------------+----------------+------------------+------------------+---------------------+--------------+------------------+-------------------+-------------+------------------+------+------------------+
|summary|       Customer ID|Product Category|     Product Price|          Quantity|Total Purchase Amount|Payment Method|      Customer Age|            Returns|Customer Name|               Age|Gender|             Churn|
+-------+------------------+----------------+------------------+------------------+---------------------+--------------+------------------+-------------------+-------------+------------------+------+------------------+
|  count|            250000|          250000|            250000|            250000|               250000|        250000|            250000|             202404|       250000|            250000|250000|            250000|
|   mean|       25004.03624|            NULL|        254.659512|          2.998896|          2725.370732|          NULL|    

In [7]:
#Renaming the column names using the ColumnRenamed
ecommerce_df = ecommerce_df.withColumnRenamed("Customer ID", "Customer_ID").withColumnRenamed("Purchase Date", "Purchase_Date")\
.withColumnRenamed("Product Category","Product_Category")\
.withColumnRenamed("Product Price","Product_Price")\
.withColumnRenamed("Total Purchase Amount","Total_Purchase_Amount")\
.withColumnRenamed("Payment Method","Payment_Method")\
.withColumnRenamed("Customer Age", "Customer_Age")\
.withColumnRenamed("Customer Name", "Customer_Name")

ecommerce_df.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+
|Customer_ID|      Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|      Customer_Name|Age|Gender|Churn|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+
|      46251|2020-09-08 09:38:32|     Electronics|           12|       3|                  740|   Credit Card|          37|    0.0|Christine Hernandez| 37|  Male|    0|
|      46251|2022-03-05 12:56:35|            Home|          468|       4|                 2739|        PayPal|          37|    0.0|Christine Hernandez| 37|  Male|    0|
|      46251|2022-05-23 18:18:01|            Home|          288|       2|                 3196|        PayPal|          37|    0.0|Christine Hernandez| 37|

In [8]:
#Changing the Datatype of the Columns
from pyspark.sql.types import IntegerType,BooleanType,DateType,DoubleType,TimestampType
ecommerce_df = ecommerce_df.withColumn("Customer_ID",ecommerce_df.Customer_ID.cast(IntegerType()))
ecommerce_df = ecommerce_df.withColumn("Purchase_Date",ecommerce_df.Purchase_Date.cast(TimestampType()))
ecommerce_df = ecommerce_df.withColumn("Product_Price",ecommerce_df.Product_Price.cast(IntegerType()))
ecommerce_df = ecommerce_df.withColumn("Quantity",ecommerce_df.Quantity.cast(IntegerType()))
ecommerce_df = ecommerce_df.withColumn("Total_Purchase_Amount",ecommerce_df.Total_Purchase_Amount.cast(IntegerType()))
ecommerce_df = ecommerce_df.withColumn("Customer_Age",ecommerce_df.Customer_Age.cast(IntegerType()))
ecommerce_df = ecommerce_df.withColumn("Returns",ecommerce_df.Returns.cast(DoubleType()))
ecommerce_df = ecommerce_df.withColumn("Churn",ecommerce_df.Churn.cast(IntegerType()))



In [9]:
ecommerce_df.printSchema()

root
 |-- Customer_ID: integer (nullable = true)
 |-- Purchase_Date: timestamp (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Product_Price: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Total_Purchase_Amount: integer (nullable = true)
 |-- Payment_Method: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Returns: double (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [10]:
ecommerce_df.show()


+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+
|Customer_ID|      Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|      Customer_Name|Age|Gender|Churn|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+
|      46251|2020-09-08 09:38:32|     Electronics|           12|       3|                  740|   Credit Card|          37|    0.0|Christine Hernandez| 37|  Male|    0|
|      46251|2022-03-05 12:56:35|            Home|          468|       4|                 2739|        PayPal|          37|    0.0|Christine Hernandez| 37|  Male|    0|
|      46251|2022-05-23 18:18:01|            Home|          288|       2|                 3196|        PayPal|          37|    0.0|Christine Hernandez| 37|

In [11]:
from pyspark.sql.functions import col
ecommerce_df.filter(ecommerce_df.Returns.isNull()).show()
ecommerce_df.filter(ecommerce_df.Customer_Age.isNull()).show()
ecommerce_df.filter(ecommerce_df.Customer_ID.isNull()).show()
ecommerce_df.filter(ecommerce_df.Purchase_Date.isNull()).show()
ecommerce_df.filter(ecommerce_df.Product_Category.isNull()).show()
ecommerce_df.filter(ecommerce_df.Product_Price.isNull()).show()
ecommerce_df.filter(ecommerce_df.Quantity.isNull()).show()
ecommerce_df.filter(ecommerce_df.Total_Purchase_Amount.isNull()).show()
ecommerce_df.filter(ecommerce_df.Payment_Method.isNull()).show()
ecommerce_df.filter(ecommerce_df.Customer_Name.isNull()).show()
ecommerce_df.filter(ecommerce_df.Age.isNull()).show()
ecommerce_df.filter(ecommerce_df.Gender.isNull()).show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+----------------+---+------+-----+
|Customer_ID|      Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|   Customer_Name|Age|Gender|Churn|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+----------------+---+------+-----+
|      28805|2020-07-02 02:54:37|        Clothing|          368|       1|                 1342|   Credit Card|          19|   NULL|    Jose Collier| 19|  Male|    0|
|      28961|2021-04-25 23:55:21|           Books|           30|       1|                 4135|        PayPal|          55|   NULL|     James Stein| 55|  Male|    0|
|      12163|2021-12-18 17:49:18|        Clothing|          255|       2|                 1642|   Credit Card|          67|   NULL|    Sonia Moreno| 67|  Male|    0|
|   

In [12]:
ecommerce_dfw = ecommerce_df.dropna(subset=['Returns'], how = 'any' )

In [13]:
from pyspark.sql.functions import current_timestamp
ecommerce_in = ecommerce_dfw.withColumn("Ingestion_Date",current_timestamp())
ecommerce_in.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+--------------------+
|Customer_ID|      Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|      Customer_Name|Age|Gender|Churn|      Ingestion_Date|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+--------------------+
|      46251|2020-09-08 09:38:32|     Electronics|           12|       3|                  740|   Credit Card|          37|    0.0|Christine Hernandez| 37|  Male|    0|2023-11-17 23:01:...|
|      46251|2022-03-05 12:56:35|            Home|          468|       4|                 2739|        PayPal|          37|    0.0|Christine Hernandez| 37|  Male|    0|2023-11-17 23:01:...|
|      46251|2022-05-23 18:18:01|            Home|

In [14]:
from datetime import date
from pyspark.sql.functions import *
ecommerce_df_final = ecommerce_in.select(ecommerce_in.Customer_ID,year(ecommerce_in.Purchase_Date).alias('Purchase_year'),\
                                         to_date(lit(ecommerce_in.Purchase_Date)).alias('Purchase_Date'),\
                                         ecommerce_in.Product_Category,ecommerce_in.Product_Price,\
                                         ecommerce_in.Quantity,ecommerce_in.Total_Purchase_Amount,\
                                         ecommerce_in.Payment_Method,ecommerce_in.Customer_Age,\
                                         ecommerce_in.Returns,ecommerce_in.Customer_Name,\
                                         ecommerce_in.Age,ecommerce_in.Gender,\
                                         ecommerce_in.Churn,ecommerce_in.Ingestion_Date)
ecommerce_df_final.show()


+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+--------------------+
|Customer_ID|Purchase_year|Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|      Customer_Name|Age|Gender|Churn|      Ingestion_Date|
+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+--------------------+
|      46251|         2020|   2020-09-08|     Electronics|           12|       3|                  740|   Credit Card|          37|    0.0|Christine Hernandez| 37|  Male|    0|2023-11-17 23:01:...|
|      46251|         2022|   2022-03-05|            Home|          468|       4|                 2739|        PayPal|          37|    0.0|Christine Hernandez| 37|  Male|    0|2023-11-17 23:01:...|
|      462

In [15]:
ecommerce_df_final.createOrReplaceTempView("rfa")
spark.sql("select count(Purchase_year) from rfa where Purchase_year = '2020'").show()

+--------------------+
|count(Purchase_year)|
+--------------------+
|               55042|
+--------------------+



In [16]:
ecommerce_df_final.show()
ecommerce_df_final.write.mode("overwrite").partitionBy('Purchase_year').parquet("/content/ecommerce/")

+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+--------------------+
|Customer_ID|Purchase_year|Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|      Customer_Name|Age|Gender|Churn|      Ingestion_Date|
+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------------+---+------+-----+--------------------+
|      46251|         2020|   2020-09-08|     Electronics|           12|       3|                  740|   Credit Card|          37|    0.0|Christine Hernandez| 37|  Male|    0|2023-11-17 23:01:...|
|      46251|         2022|   2022-03-05|            Home|          468|       4|                 2739|        PayPal|          37|    0.0|Christine Hernandez| 37|  Male|    0|2023-11-17 23:01:...|
|      462

In [18]:
ecommerce_2020= spark.read.format('parquet').option('header',True).load("/content/ecommerce/Purchase_year=2020/part-00000-16ade1af-e9c2-463e-87c3-6e0733e67117.c000.snappy.parquet")
ecommerce_1= spark.read.format('parquet').option('header',True).load("/content/ecommerce/Purchase_year=2020/part-00001-16ade1af-e9c2-463e-87c3-6e0733e67117.c000.snappy.parquet")
ecommerce_2020.createOrReplaceTempView("emr")
ecommerce_1.createOrReplaceTempView("ems")
emc_df = spark.sql("select * from emr union all select * from ems")
emc_df.createOrReplaceTempView("emc_2020")


#Goal_1
###*Analyze the dataset to gain insights into specific customer demographics, purchase frequency, and product preferences.*

In [19]:
spark.sql("select Customer_ID,Purchase_Date, Product_Category, Customer_Name, Churn,\
 Lag(Purchase_Date) over(order by Purchase_Date) as previous_row, \
 Purchase_Date - Lag(Purchase_Date) over(order by Purchase_Date) as difference\
  from emc_2020 where Customer_ID = 33906 ").show()

+-----------+-------------+----------------+--------------+-----+------------+-----------------+
|Customer_ID|Purchase_Date|Product_Category| Customer_Name|Churn|previous_row|       difference|
+-----------+-------------+----------------+--------------+-----+------------+-----------------+
|      33906|   2020-03-01|        Clothing|Aaron Bautista|    0|        NULL|             NULL|
|      33906|   2020-04-07|           Books|Aaron Bautista|    0|  2020-03-01|INTERVAL '37' DAY|
|      33906|   2020-05-02|           Books|Aaron Bautista|    0|  2020-04-07|INTERVAL '25' DAY|
|      33906|   2020-06-23|            Home|Aaron Bautista|    0|  2020-05-02|INTERVAL '52' DAY|
|      33906|   2020-07-25|            Home|Aaron Bautista|    0|  2020-06-23|INTERVAL '32' DAY|
+-----------+-------------+----------------+--------------+-----+------------+-----------------+



#Goal_2
###*Exploring the factors contributing to customer churn.*

In [20]:
spark.sql("select Customer_ID,Purchase_Date, Product_Category, Customer_Name, Churn,\
 Lag(Purchase_Date) over(order by Purchase_Date) as previous_row, \
 Purchase_Date - Lag(Purchase_Date) over(order by Purchase_Date) as difference\
  from rfa where Customer_ID = 28248 ").show()

+-----------+-------------+----------------+-------------+-----+------------+------------------+
|Customer_ID|Purchase_Date|Product_Category|Customer_Name|Churn|previous_row|        difference|
+-----------+-------------+----------------+-------------+-----+------------+------------------+
|      28248|   2020-04-26|           Books|Kelly Ramirez|    1|        NULL|              NULL|
|      28248|   2021-06-25|            Home|Kelly Ramirez|    1|  2020-04-26|INTERVAL '425' DAY|
|      28248|   2021-08-03|     Electronics|Kelly Ramirez|    1|  2021-06-25| INTERVAL '39' DAY|
|      28248|   2022-09-22|           Books|Kelly Ramirez|    1|  2021-08-03|INTERVAL '415' DAY|
|      28248|   2023-09-12|            Home|Kelly Ramirez|    1|  2022-09-22|INTERVAL '355' DAY|
+-----------+-------------+----------------+-------------+-----+------------+------------------+



In [21]:
spark.sql("select * from rfa where Customer_ID = 28248 order by Purchase_Date").show()

+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------+---+------+-----+--------------------+
|Customer_ID|Purchase_year|Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|Customer_Name|Age|Gender|Churn|      Ingestion_Date|
+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------+---+------+-----+--------------------+
|      28248|         2020|   2020-04-26|           Books|           70|       5|                 1037|        PayPal|          48|    0.0|Kelly Ramirez| 48|  Male|    1|2023-11-17 23:03:...|
|      28248|         2021|   2021-06-25|            Home|          131|       4|                 2019|        PayPal|          48|    0.0|Kelly Ramirez| 48|  Male|    1|2023-11-17 23:03:...|
|      28248|         2021|   2021-08-03

In [22]:
spark.sql("select * from rfa where Customer_ID = 3396 order by Purchase_Date").show()

+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------+---+------+-----+--------------------+
|Customer_ID|Purchase_year|Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|Customer_Name|Age|Gender|Churn|      Ingestion_Date|
+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+-------------+---+------+-----+--------------------+
|       3396|         2020|   2020-08-28|           Books|          391|       5|                 1354|        PayPal|          68|    0.0|  Mary Kelley| 68|Female|    0|2023-11-17 23:03:...|
|       3396|         2021|   2021-05-20|     Electronics|          406|       3|                 1187|   Credit Card|          68|    1.0|  Mary Kelley| 68|Female|    0|2023-11-17 23:03:...|
|       3396|         2022|   2022-03-24

In [23]:
mrf = spark.sql("select Distinct(Customer_ID) from rfa where Churn = 0")
mrf.createOrReplaceTempView('red')

In [24]:
mpf = spark.sql("select Distinct(Customer_ID) from rfa where Churn = 1")
mpf.createOrReplaceTempView('green')

In [25]:
spark.sql("select Customer_ID from red intersect select Customer_ID from green").show()

+-----------+
|Customer_ID|
+-----------+
+-----------+



#Goal_3
###*Analyze the returns of the customer over the years based upon the product category.*

In [26]:
spark.sql("select count(*) from rfa where Product_Category = 'Books'").show()
spark.sql("select * from rfa where Product_Category = 'Books' and Returns = 1").show(50)
spark.sql("select * from rfa where Product_Category = 'Books' and Returns = 0").show(50)

+--------+
|count(1)|
+--------+
|   60595|
+--------+

+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+------------------+---+------+-----+--------------------+
|Customer_ID|Purchase_year|Purchase_Date|Product_Category|Product_Price|Quantity|Total_Purchase_Amount|Payment_Method|Customer_Age|Returns|     Customer_Name|Age|Gender|Churn|      Ingestion_Date|
+-----------+-------------+-------------+----------------+-------------+--------+---------------------+--------------+------------+-------+------------------+---+------+-----+--------------------+
|      28805|         2021|   2021-01-18|           Books|          348|       1|                 2682|   Credit Card|          19|    1.0|      Jose Collier| 19|  Male|    0|2023-11-17 23:03:...|
|      28961|         2021|   2021-09-10|           Books|          489|       3|                 2213|   Credit Card|          55|    1.0|       James Stei

# Goal 4
###*Analyzing the revenue in each category over the years*

In [44]:
#Total Product sales over the years
spark.sql("select Product_Category, SUM(Product_Price) as Sales from rfa group by Product_Category").show()

+----------------+--------+
|Product_Category|   Sales|
+----------------+--------+
|            Home|10289915|
|     Electronics|10349310|
|        Clothing|15415042|
|           Books|15432741|
+----------------+--------+



In [45]:
#Product sales in year 2020
spark.sql("select Product_Category, SUM(Product_Price) as Sales from rfa where Purchase_year = 2020 group by Product_Category").show()

+----------------+-------+
|Product_Category|  Sales|
+----------------+-------+
|            Home|2795338|
|     Electronics|2809461|
|        Clothing|4158653|
|           Books|4203745|
+----------------+-------+



In [46]:
# Product sales in year 2021
spark.sql("select Product_Category, SUM(Product_Price) as Sales from rfa where Purchase_year = 2021 group by Product_Category").show()

+----------------+-------+
|Product_Category|  Sales|
+----------------+-------+
|            Home|2814963|
|     Electronics|2813062|
|        Clothing|4156609|
|           Books|4099013|
+----------------+-------+



In [47]:
#Product sales in year 2022
spark.sql("select Product_Category, SUM(Product_Price) as Sales from rfa where Purchase_year = 2022 group by Product_Category").show()

+----------------+-------+
|Product_Category|  Sales|
+----------------+-------+
|            Home|2743828|
|     Electronics|2762286|
|        Clothing|4141310|
|           Books|4116428|
+----------------+-------+



In [48]:
#Product sales in year 20233
spark.sql("select Product_Category, SUM(Product_Price) as Sales from rfa where Purchase_year = 2023 group by Product_Category").show()

+----------------+-------+
|Product_Category|  Sales|
+----------------+-------+
|            Home|1935786|
|     Electronics|1964501|
|        Clothing|2958470|
|           Books|3013555|
+----------------+-------+



#Goal_5
###*Analyszing the product sales based upon the customer age groups*

In [61]:
spark.sql("select Product_Category, Customer_Age, SUM(Product_Price) as Sales from rfa group by Product_Category, Customer_Age order by Product_Category, Customer_Age").show(100)

+----------------+------------+------+
|Product_Category|Customer_Age| Sales|
+----------------+------------+------+
|           Books|          18|311523|
|           Books|          19|300083|
|           Books|          20|277736|
|           Books|          21|322476|
|           Books|          22|293266|
|           Books|          23|289077|
|           Books|          24|280115|
|           Books|          25|291175|
|           Books|          26|269283|
|           Books|          27|298864|
|           Books|          28|296725|
|           Books|          29|296411|
|           Books|          30|300448|
|           Books|          31|294492|
|           Books|          32|270264|
|           Books|          33|303621|
|           Books|          34|292898|
|           Books|          35|271999|
|           Books|          36|283290|
|           Books|          37|299425|
|           Books|          38|295426|
|           Books|          39|289097|
|           Books|       